In [ ]:
! pip install -qU memorizz yahooquery openai voyageai

# ToolBox Functionality in the MemoRizz Library

-------

The Toolbox system in MemoRizz provides a powerful framework for registering, managing, and utilizing external functions as AI-callable tools. This system enables agents to seamlessly interact with external systems, APIs, and data sources through a standardized interface.

## Core Concepts

### Function Registration and Discovery
- **Registration**: Convert Python functions into LLM-callable tools with semantic metadata
- **Vector Embeddings**: Index tools for efficient semantic retrieval based on natural language queries
- **Discoverability**: Find relevant tools based on contextual requirements without explicit references 
- **Persistence**: Store tool definitions across sessions and application restarts

### Tool Management
- **Centralized Repository**: Maintain all tools in a single, organized collection
- **Access Control**: Configure private or global tool access patterns for agents
- **Tool Metadata**: Enhance functions with rich descriptions and usage examples
- **Versioning**: Update tool implementations while maintaining consistent identifiers

### Agent Integration
- **Dynamic Tool Selection**: Agents can discover and utilize the most relevant tools for each query
- **Contextual Relevance**: Tools are matched to queries based on semantic similarity
- **Automatic Documentation**: Generate parameter schemas and usage instructions for LLMs
- **Error Handling**: Gracefully manage missing or broken tool implementations

## Implementation Details

Under the hood, the Toolbox system:
1. Analyzes function signatures and docstrings to extract parameter information
2. Generates embeddings that capture the semantic meaning of each tool
3. Stores both the metadata and function references in the memory provider
4. Provides retrieval mechanisms for both exact and similarity-based lookups

This architecture enables a powerful "tools as a service" model where functions can be registered once and utilized by multiple agents throughout your application ecosystem.


## Implementation & Usage Guidance


To fully leverage MemoRizz's memory subsystem, always initialize a `MemoryProvider` before interacting with the library. In this workflow, the `MemoryProvider` acts as the authoritative store for all `ToolBox` data—handling both persistence and retrieval of function definitions, tool metadata, and embedding vectors throughout your application lifecycle. 

The ToolBox component relies on this persistent storage to enable seamless tool discovery, contextual matching, and function execution across different sessions and application instances.


In [1]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [2]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [3]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [4]:
set_env_securely("VOYAGE_API_KEY", "Enter your VOYAGE_API_KEY API Key: ")

In [5]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a mongodb config with voyageai embeddings
mongodb_config = MongoDBConfig(
    uri=os.environ["MONGODB_URI"],
    db_name="testing_memorizz",
    embedding_provider="voyageai",
    embedding_config={
        "embedding_type": "contextualized",
        "model": "voyage-context-3",
        "output_dimension": 256,
    }
)

# Create a memory provider
memory_provider = MongoDBProvider(mongodb_config)

/Users/richmondalake/miniconda3/envs/agent_memory_course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector search index 'vector_index' already exists for collection personas.
Vector search index 'vector_index' already exists for collection toolbox.
Vector search index 'vector_index' already exists for collection short_term_memory.
Vector search index 'vector_index' already exists for collection long_term_memory.
Vector search index 'vector_index' already exists for collection conversation_memory.
Vector search index 'vector_index' already exists for collection workflow_memory.
Vector search index 'vector_index' already exists for collection agents.
Vector search index 'vector_index' already exists for collection shared_memory.
Vector search index 'vector_index' already exists for collection summaries.


### Step 1: Creating Function-Based Tools

In this initial step, we define specialized Python functions that will serve as the foundation for our toolbox. These functions encapsulate discrete capabilities that can be semantically discovered and executed by AI agents:

1. The `get_stock_price()` function interfaces with the `yfinance` library to retrieve real-time financial market data. It accepts a stock symbol parameter (e.g., 'AAPL') and an optional currency code, returning formatted price information with proper numerical formatting.

2. The `get_weather()` function leverages the Open-Meteo API to access meteorological data based on precise geolocation coordinates. It extracts the current temperature at the specified latitude and longitude, providing environmental context to agent interactions.

These functions are designed with comprehensive type hints, descriptive docstrings, and robust error handling to ensure they can be properly indexed, discovered, and reliably executed when registered in the MemoRizz toolbox system.


In [6]:
from functools import lru_cache
from yahooquery import Ticker
import time

@lru_cache(maxsize=128)
def _fetch_price(symbol: str) -> float:
    """
    Internal helper to fetch the latest market price via yahooquery.
    Caching helps avoid repeated hits for the same symbol.
    """
    ticker = Ticker(symbol)
    # This returns a dict keyed by symbol:
    info = ticker.price or {}
    # regularMarketPrice holds the current trading price
    price = info.get(symbol.upper(), {}).get("regularMarketPrice")
    if price is None:
        raise ValueError(f"No price data for '{symbol}'")
    return price

def get_stock_price(
    symbol: str,
    currency: str = "USD",
    retry: int = 3,
    backoff: float = 0.5
) -> str:
    """
    Get the current stock price for a given symbol using yahooquery,
    with simple retry/backoff to handle occasional rate-limits.

    Parameters
    ----------
    symbol : str
        Stock ticker, e.g. "AAPL"
    currency : str, optional
        Currency code (Currently informational only; yahooquery returns native)
    retry : int, optional
        Number of retries on failure (default: 3)
    backoff : float, optional
        Backoff factor in seconds between retries (default: 0.5s)

    Returns
    -------
    str
        e.g. "The current price of AAPL is 172.34 USD."
    """
    symbol = symbol.upper()
    last_err = None
    for attempt in range(1, retry + 1):
        try:
            price = _fetch_price(symbol)
            return f"The current price of {symbol} is {price:.2f} {currency.upper()}."
        except Exception as e:
            last_err = e
            # simple backoff
            time.sleep(backoff * attempt)
    # if we get here, all retries failed
    raise RuntimeError(f"Failed to fetch price for '{symbol}' after {retry} attempts: {last_err}")


In [7]:
print(get_stock_price("AAPL"))

The current price of AAPL is 232.01 USD.


In [8]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [9]:
latitude = 40.7128
longitude = -74.0060
weather = get_weather(latitude, longitude)
print(weather)

30.8


Ensure values from the methods are printed in the cells above


### Step 2: Initializing the Toolbox and Registering Function Tools

This phase focuses on transforming our raw Python functions into semantically discoverable AI tools through the MemoRizz orchestration layer:

1. Instantiate a Toolbox instance that interfaces with our memory provider
2. Register each function as a discoverable tool within the system
3. Generate embedding vectors and metadata to enable contextual discovery
4. Persist the registered tools in the memory provider for cross-session availability

The registration process automatically analyzes function signatures, docstrings, and behavior patterns to create comprehensive tool schemas that language models can understand and invoke during agent interactions.


In [10]:
from memorizz.long_term_memory.procedural.toolbox import Toolbox

toolbox = Toolbox(memory_provider=memory_provider)

# Now the tools are registered in the memory provider within the toolbox
toolbox.register_tool(get_weather)
toolbox.register_tool(get_stock_price)

'689caba8d37d383690b36265'


### Step 3: Retrieving Tools from the Toolbox Repository

The MemoRizz toolbox provides multiple access patterns for tool retrieval, accommodating different operational needs:

1. **Deterministic Retrieval by Name**: The `get_tool_by_name()` method enables direct access to specific tools using their registered function names. This approach is ideal when you know exactly which tool you need.

2. **Retrieval by Unique Identifier**: The `get_tool_by_id()` method allows precise tool access using the unique ID assigned during registration. This method is particularly useful in distributed systems or when tool names might overlap across different domains.

3. **Semantic Discovery**: The `get_most_similar_tools()` method leverages vector embeddings to identify contextually relevant tools based on natural language queries. This powerful capability allows agents to discover appropriate tools without knowing their exact names.

4. **Comprehensive Inventory**: The `list_tools()` method provides a complete catalog of all registered tools in the toolbox repository, enabling auditing, documentation generation, and management operations.

Each retrieval method returns both the tool metadata (parameters, descriptions, etc.) and the callable Python function reference, ensuring seamless integration between the semantic discovery layer and executable functionality.


In [11]:
specific_tool = toolbox.get_tool_by_name("get_stock_price")
specific_tool

In [12]:
# Get similar tools
similar_tools = toolbox.get_most_similar_tools("Get the stock price for Apple", limit=1)
similar_tools

[{'_id': ObjectId('6894ecadcb348e000d054e7d'),
  'function': {'name': 'updated_get_stock_price',
   'description': 'Retrieve the current stock price for a given ticker symbol, with options for specifying the currency. This function is useful for investors or analysts looking to obtain real-time market data in different currencies.',
   'parameters': [{'name': 'ticker',
     'description': "The stock ticker symbol represents a specific company's stock, such as 'AAPL' for Apple Inc. This parameter is required and must be a valid ticker symbol.",
     'type': 'string',
     'required': True},
    {'name': 'currency',
     'description': "Specifies the currency in which the stock price should be returned. The default value is 'USD' (U.S. Dollar). This parameter is optional and should be a valid currency code as per ISO 4217 standards.",
     'type': 'string',
     'required': False}],
   'required': ['ticker'],
   'queries': ["What is the current stock price for 'GOOGL' in EUR?",
    "Get 

In [13]:
list_all_tools = toolbox.list_tools()
list_all_tools

[{'_id': ObjectId('6894ecadcb348e000d054e7d'),
  'function': {'name': 'updated_get_stock_price',
   'description': 'Retrieve the current stock price for a given ticker symbol, with options for specifying the currency. This function is useful for investors or analysts looking to obtain real-time market data in different currencies.',
   'parameters': [{'name': 'ticker',
     'description': "The stock ticker symbol represents a specific company's stock, such as 'AAPL' for Apple Inc. This parameter is required and must be a valid ticker symbol.",
     'type': 'string',
     'required': True},
    {'name': 'currency',
     'description': "Specifies the currency in which the stock price should be returned. The default value is 'USD' (U.S. Dollar). This parameter is optional and should be a valid currency code as per ISO 4217 standards.",
     'type': 'string',
     'required': False}],
   'required': ['ticker'],
   'queries': ["What is the current stock price for 'GOOGL' in EUR?",
    "Get 

In [14]:
# Get a tool by id
tool_id = similar_tools[0]["_id"]
tool_retrieved_by_id = toolbox.get_tool_by_id(tool_id)
tool_retrieved_by_id

{'_id': ObjectId('6894ecadcb348e000d054e7d'),
 'function': {'name': 'updated_get_stock_price',
  'description': 'Retrieve the current stock price for a given ticker symbol, with options for specifying the currency. This function is useful for investors or analysts looking to obtain real-time market data in different currencies.',
  'parameters': [{'name': 'ticker',
    'description': "The stock ticker symbol represents a specific company's stock, such as 'AAPL' for Apple Inc. This parameter is required and must be a valid ticker symbol.",
    'type': 'string',
    'required': True},
   {'name': 'currency',
    'description': "Specifies the currency in which the stock price should be returned. The default value is 'USD' (U.S. Dollar). This parameter is optional and should be a valid currency code as per ISO 4217 standards.",
    'type': 'string',
    'required': False}],
  'required': ['ticker'],
  'queries': ["What is the current stock price for 'GOOGL' in EUR?",
   "Get the stock pric


### Creating a MemAgent with a Toolbox

#### Design Considerations

- A tool added via the `add_tool` method to the MemAgent will be located in the `tools` attribute of the agent itself. This approach ensures each agent maintains its own isolated tool registry.

- A copy of the tool in the toolbox is made and stored within the MemAgent, preserving the tool's metadata, schema, and function reference. This design prevents cross-agent contamination when tool implementations are updated.

- Methods such as `refresh_tools(tool_id)` allow you to update specific tools within an agent when their implementations change in the source toolbox. This ensures agents can receive tool updates without complete reconfiguration.

- The `tool_access` attribute determines how tools are discovered during agent execution:
  - When set to `"private"` (default), agents only access tools explicitly added to them
  - When set to `"global"`, agents can dynamically discover any tool in the toolbox based on query relevance

- Each agent maintains a private map of `tool_id → python_function` references in the `_tool_functions` attribute, ensuring function calls are properly directed to their implementations.

- When an agent is persisted via `save()`, tool references are stored in the database, making them available after application restarts or when loading agents on different machines.

- Tools with missing implementations are gracefully handled during agent execution with informative error messages rather than runtime failures.

- You can inspect an agent's available tools using the native `list_tools()` method inherited from the toolbox interface.

This architecture provides a clean separation between tool definition (in the toolbox) and tool availability (in the agent), enabling fine-grained control over which capabilities are exposed to different agents in your system.


Create a MemAgent and pass in the memory provider that the agent is stored in

In [15]:
from memorizz import MemAgent

monday_agent = MemAgent(memory_provider=memory_provider)

In [16]:
# Save the agent to the memory provider
monday_agent.save()

MemAgent(agent_id=689cabb4d37d383690b36266, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x117963b50>)


The `add_tool` method can add tools to the Agent document in two ways:

1. **Individual Tool Registration**: By providing a specific `tool_id` parameter, agents can selectively incorporate a single tool from the toolbox. This approach gives precise control over which capabilities are available to each agent.

2. **Bulk Toolbox Integration**: By passing an entire `toolbox` instance, agents can automatically import all compatible tools in a single operation. This method is efficient when you want an agent to have access to a predefined set of related capabilities.

In both cases, the system handles:
- Resolving tool metadata from the memory provider
- Connecting to the actual Python function implementations
- Ensuring the tools are properly formatted for LLM function-calling
- Persisting the tool references in the agent's configuration

This flexibility allows developers to create specialized agents with targeted functionality or general-purpose agents with comprehensive tool access, all while maintaining a clean separation between tool definition and agent configuration.


In [17]:

monday_agent.add_tool(toolbox=toolbox)

True

In [18]:
monday_agent

MemAgent(agent_id=689cabb4d37d383690b36266, memory_provider=<memorizz.memory_provider.mongodb.provider.MongoDBProvider object at 0x117963b50>)

In [19]:
monday_agent.run("Get me the stock price of Apple")

'The current stock price of Apple (AAPL) is $232.01 USD. If you need more details or a live update, let me know!'

In [20]:
monday_agent.run("Get me the weather in London")

'The current weather in London is 25.7°C. If you need additional details such as humidity, wind speed, or a forecast for the coming days, just let me know!'

### Step 4: Updating Tools

In [21]:
updated_tool = toolbox.update_tool_by_id(tool_id, {"name": "get_stock_price_for_american_companies"})
updated_tool

True

In [22]:
# Confirm that the tool has been updated
toolbox.get_tool_by_id(tool_id)

{'_id': ObjectId('6894ecadcb348e000d054e7d'),
 'function': {'name': 'updated_get_stock_price',
  'description': 'Retrieve the current stock price for a given ticker symbol, with options for specifying the currency. This function is useful for investors or analysts looking to obtain real-time market data in different currencies.',
  'parameters': [{'name': 'ticker',
    'description': "The stock ticker symbol represents a specific company's stock, such as 'AAPL' for Apple Inc. This parameter is required and must be a valid ticker symbol.",
    'type': 'string',
    'required': True},
   {'name': 'currency',
    'description': "Specifies the currency in which the stock price should be returned. The default value is 'USD' (U.S. Dollar). This parameter is optional and should be a valid currency code as per ISO 4217 standards.",
    'type': 'string',
    'required': False}],
  'required': ['ticker'],
  'queries': ["What is the current stock price for 'GOOGL' in EUR?",
   "Get the stock pric

In [23]:
# Delete a tool by name
success = toolbox.delete_tool_by_name("get_stock_price")
success

False

In [24]:
# Delete a tool by id
success_deleting_tool_by_id = toolbox.delete_tool_by_id(tool_id)
success_deleting_tool_by_id


True

In [25]:
# Delete all tools within the toolbox
success_deleting_all_tools = toolbox.delete_all()
success_deleting_all_tools

True

### Step  : Augmenting Tool Metadata

You can enhance the tool's searchability and retrieval accuracy by using an LLM to augment the tool description and generate synthetic queries. These enhancements are automatically added to the tool's metadata and used during embedding generation, significantly improving the tool selection process.

To enable this feature, simply set `augment=True` when registering your tool using the `@toolbox.register_tool` decorator.

In [26]:
@toolbox.register_tool(augment=True)
def updated_get_stock_price(ticker: str, currency: str = "USD") -> str:
    """
    Get the current stock price for a specified ticker symbol.

    Parameters:
    -----------
    ticker : str
        The stock ticker symbol (e.g., "AAPL" for Apple).
    currency : str, optional
        The currency in which the price should be returned (default is USD).

    Returns:
    --------
    str
        A description of the current stock price.
    """
    # Implementation here
    pass